# (worm)CellPLATO | Cell Plasticity Analysis Tool

In [1]:
'cellinf' is a json file containing information about the cells, their names, lineage, cell type etc. 

'gene_CPM' is cell x gene x time matrix

'gene_names' is a list of the corresponding gene names

'gene_time' the time sampling for gene and position data is different. This vector contains the timestamps for the gene data. 

'pos_rpz' 3 x cell x time matrix  containing the cell positions in the cylindrical coordinate system. r is radial distance, p is the angular distance and z is the distance along the A-P axis.

'pos_time' timestamp for the position data. 

'pos_xyz' 3 x cell x time matrix with the cartesian position of each cell over time.

SyntaxError: invalid syntax (3207632024.py, line 1)

Step 1: Fill in the config file!

<div class="alert alert-block alert-danger">
Set your kernel to 'cellPLATO' before continuing
</div>

<div class="alert alert-block alert-success">
<h2>1. Start by importing packages for cellPLATO</h1>
</div>

This includes cellPLATO itself, and all of the modules you will need

* Import these packages, checking that you have them
* We're also importing a lot of the modules in cellPLATO, if this cell runs successfully, you are good to go!

In [1]:
import cellPLATO as cp

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import imageio

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px

OVERWRITE_DATAFRAMES = True

Finished running cellPLATO initialization and loaded config.
Initializing:  WormData
Hypthesis testing using:  st.ttest_ind
Plots will be exported to:  D:/Michael_Shannon/Hari_Collab/test_data_OUTPUT/WormData\2024-02-21_17-11-10-644365\plots/
Using unique embedding per dataset shortname:  WormData
Exporting static Superplots
Exporting static Superplots
Exporting static Plots of Differences
Exporting static Marginal scatterplots
Exporting static Timeplots
Exporting Bar plots
Exporting SNS Bar plots
Using corresponding CTL_SHORTLABEL:  Worm_one  for condition:  Condition_one
Dataset in current notebook:  WormData
Finished initializing data_processing



Bad key text.latex.preview in file c:\ProgramData\Anaconda3\envs\cellPLATO_gitversion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file c:\ProgramData\Anaconda3\envs\cellPLATO_gitversion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file c:\ProgramData\Anaconda3\envs\cellPLATO_gitversion\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_qualit

Finished intializing visualizations
Finished initializing cellPLATO


In [2]:
import os
import h5py

master_directory = cp.DATA_PATH

# Iterate over the folders in the master directory
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    
    # Check if the folder is a directory
    if os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            # Check if the file is an h5 file
            if file.endswith('.h5'):
                # Open the h5 file
                with h5py.File(file_path, 'r') as f:
                    # Print the keys of the h5 file
                    print(f.keys())


<KeysViewHDF5 ['cellinf', 'gene_CPM', 'gene_names', 'gene_time', 'pos_rpz', 'pos_time', 'pos_xyz']>


'cellinf' is a json file containing information about the cells, their names, lineage, cell type etc. 

'gene_CPM' is cell x gene x time matrix

'gene_names' is a list of the corresponding gene names

'gene_time' the time sampling for gene and position data is different. This vector contains the timestamps for the gene data. 

'pos_rpz' 3 x cell x time matrix  containing the cell positions in the cylindrical coordinate system. r is radial distance, p is the angular distance and z is the distance along the A-P axis.

'pos_time' timestamp for the position data. 

'pos_xyz' 3 x cell x time matrix with the cartesian position of each cell over time.

### STEP 0: This is a check cell

In [19]:
import h5py
import numpy as np

master_directory = cp.DATA_PATH

# Placeholder for the last read h5 file path
h5_file_path = None

# Iterate and find the last h5 file for demonstration
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)

with h5py.File(h5_file_path, 'r') as f:
    # Load the dataset
    pos_rpz = f['pos_rpz'][()]
    
    # 1. Check the shape
    print("Shape of pos_rpz:", pos_rpz.shape)
    
    # The shape should be (3, num_cells, num_times)
    
    # 2. Inspect the data
    # Viewing a small portion to understand its arrangement
    # For example, let's view the first cell's data at the first time point
    print("Sample data (first cell, first time point):")
    print("r (radial distance):", pos_rpz[0, 0, 0])
    print("p (angular distance):", pos_rpz[1, 0, 0])
    print("z (distance along A-P axis):", pos_rpz[2, 0, 0])
    
    # Verify that the first dimension indeed corresponds to r, p, z for a sanity check
    if pos_rpz.shape[0] != 3:
        print("Warning: The first dimension does not match the expected size for r, p, z.")
    else:
        print("The first dimension correctly represents r, p, z coordinates.")
    
    # Additional checks can be done here depending on what specific issues you're concerned about


Shape of pos_rpz: (331, 321, 3)
Sample data (first cell, first time point):
r (radial distance): 3.092721700668335
p (angular distance): 3.0966713428497314
z (distance along A-P axis): 3.096306324005127


## STEP 1: load everything from the h5, doing a special JSON load for the cellinf

In [2]:
import h5py
import numpy as np
import json

master_directory = cp.DATA_PATH

# Placeholder for the last read h5 file path
h5_file_path = None

# Iterate and find the last h5 file for demonstration
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)

with h5py.File(h5_file_path, 'r') as f:
    # Load the datasets
    pos_rpz = f['pos_rpz'][()]
    # cellinf = f['cellinf'][()]
    gene_CPM = f['gene_CPM'][()]
    gene_names = f['gene_names'][()] # time, gene expression level, cell
    gene_time = f['gene_time'][()] # A vector of 130 elements which is the REAL time sampling for the gene data
    pos_xyz = f['pos_xyz'][()]
    pos_time = f['pos_time'][()] # A vector of 331 elements which is the REAL time sampling for the position data.


    # This is special because it is a byte string from a JSON object

    if 'cellinf' in f:
        # Extract the dataset
        json_data_array = f['cellinf'][()]

        # Given the shape is (1, 1), access the nested byte string
        if json_data_array.size > 0:
            json_bytes = json_data_array[0, 0]  # Access the byte string

            # Decode the byte string to get the JSON string
            json_str = json_bytes.decode('utf-8')
            
            # Parse the JSON string into a Python object
            cell_info = json.loads(json_str)
            
            # Print the first few items to check
            print(cell_info[:5])
        else:
            print("The 'cellinf' dataset appears to be empty.")
    else:
        print("'cellinf' dataset does not exist in the HDF5 file.")



[{'lineage_id': 'ABpraappaa', 'name': 'hyp4', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABplaappaa', 'name': 'hyp4', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABplaaapaa', 'name': 'XXXL', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABarpappaa', 'name': 'XXXR', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABarapaaapa', 'name': 'm1VR', 'type': 'muscle', 'cell_death': False, 'isterminal': True}]


## STEP 2: make the dataframe from the radial coordinates

In [3]:
import h5py
import pandas as pd
import numpy as np
import os

master_directory = cp.DATA_PATH

# Placeholder for the last read h5 file path
h5_file_path = None

# Iterate and find the last h5 file for demonstration
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)

if h5_file_path:
    with h5py.File(h5_file_path, 'r') as f:
        pos_rpz = f['pos_rpz'][()]  # Assuming shape is (3, num_cells, num_times)

    # Since the last dimension is the coordinates, we can reshape directly
    num_cells, num_times = pos_rpz.shape[0], pos_rpz.shape[1]
    # Flatten the array while keeping the r, p, z coordinates intact
    data_reshaped = pos_rpz.reshape(num_cells * num_times, 3)

    # Create DataFrame
    df_pos_rpz = pd.DataFrame(data_reshaped, columns=['r', 'p', 'z'])

    # Generate cell_id and time_id
    cell_ids = np.repeat(np.arange(num_cells), num_times)
    time_ids = np.tile(np.arange(num_times), num_cells)

    # Add cell_id and time_id to the DataFrame
    df_pos_rpz['cell_id'] = cell_ids
    df_pos_rpz['time_id'] = time_ids

    # Display the first few rows of the DataFrame
    print(df_pos_rpz.head())

          r         p          z  cell_id  time_id
0  3.092722 -1.208199  15.028050        0        0
1  3.148723 -1.970752  15.410750        0        1
2  4.094421 -2.825229  16.787140        0        2
3  3.943589 -0.301919  18.441759        0        3
4  2.134434 -2.099260  21.032261        0        4


### STEP 3: Map the contents of cell_info to the dataframe and add them in (cell names)

In [4]:
########################## ADDING THE JSON INFO TO THE DATAFRAME ##############################

# Assuming cell_info and df_pos_rpz are already defined and populated as before

# Extend the cell_name_mapping to include additional attributes
cell_attributes_mapping = {i: {'name': cell['name'], 
                               'lineage_id': cell['lineage_id'], 
                               'type': cell['type'], 
                               'cell_death': cell['cell_death'], 
                               'isterminal': cell['isterminal']} 
                           for i, cell in enumerate(cell_info)}

# Function to safely get attributes for each cell_id
def get_cell_attribute(cell_id, attribute):
    return cell_attributes_mapping.get(cell_id, {}).get(attribute, 'Unknown')

# Now, map each cell_id in your DataFrame to its corresponding attributes
df_pos_rpz['lineage_id'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'lineage_id'))
df_pos_rpz['cell_type'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'type'))
df_pos_rpz['cell_death'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'cell_death'))
df_pos_rpz['isterminal'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'isterminal'))

# Now, df_pos_rpz includes additional columns for lineage_id, cell_type, cell_death, and isterminal
print(df_pos_rpz.head())


          r         p          z  cell_id  time_id  lineage_id cell_type  \
0  3.092722 -1.208199  15.028050        0        0  ABpraappaa  hypoderm   
1  3.148723 -1.970752  15.410750        0        1  ABpraappaa  hypoderm   
2  4.094421 -2.825229  16.787140        0        2  ABpraappaa  hypoderm   
3  3.943589 -0.301919  18.441759        0        3  ABpraappaa  hypoderm   
4  2.134434 -2.099260  21.032261        0        4  ABpraappaa  hypoderm   

  cell_death isterminal  
0      False       True  
1      False       True  
2      False       True  
3      False       True  
4      False       True  


In [5]:
df_pos_rpz

,r,p,z,cell_id,time_id,lineage_id,cell_type,cell_death,isterminal
0,3.092722,-1.208199e+00,15.028050,0,0,ABpraappaa,hypoderm,False,True
1,3.148723,-1.970752e+00,15.410750,0,1,ABpraappaa,hypoderm,False,True
2,4.094421,-2.825229e+00,16.787140,0,2,ABpraappaa,hypoderm,False,True
3,3.943589,-3.019188e-01,18.441759,0,3,ABpraappaa,hypoderm,False,True
4,2.134434,-2.099260e+00,21.032261,0,4,ABpraappaa,hypoderm,False,True
...,...,...,...,...,...,...,...,...,...
106246,1.976188,-5.053166e-16,184.381393,330,316,Unknown,Unknown,Unknown,Unknown
106247,1.536988,1.591760e+00,185.985107,330,317,Unknown,Unknown,Unknown,Unknown
106248,0.902613,-1.439479e+00,189.487793,330,318,Unknown,Unknown,Unknown,Unknown
106249,0.538817,5.551305e-01,185.632797,330,319,Unknown,Unknown,Unknown,Unknown


In [55]:
# # THIS GETS THE JSON STUFF

# import h5py
# import json

# master_directory = cp.DATA_PATH

# # Placeholder for the last read h5 file path
# h5_file_path = None

# # Iterate and find the last h5 file for demonstration
# for folder in os.listdir(master_directory):
#     folder_path = os.path.join(master_directory, folder)
#     if os.path.isdir(folder_path):
#         for file in os.listdir(folder_path):
#             h5_file_path = os.path.join(folder_path, file)



# with h5py.File(h5_file_path, 'r') as f:
#     # Check if 'cellinf' exists in the file
#     if 'cellinf' in f:
#         # Extract the dataset
#         json_data_array = f['cellinf'][()]

#         # Given the shape is (1, 1), access the nested byte string
#         if json_data_array.size > 0:
#             json_bytes = json_data_array[0, 0]  # Access the byte string

#             # Decode the byte string to get the JSON string
#             json_str = json_bytes.decode('utf-8')
            
#             # Parse the JSON string into a Python object
#             cell_info = json.loads(json_str)
            
#             # Print the first few items to check
#             print(cell_info[:5])
#         else:
#             print("The 'cellinf' dataset appears to be empty.")
#     else:
#         print("'cellinf' dataset does not exist in the HDF5 file.")





[{'lineage_id': 'ABpraappaa', 'name': 'hyp4', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABplaappaa', 'name': 'hyp4', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABplaaapaa', 'name': 'XXXL', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABarpappaa', 'name': 'XXXR', 'type': 'hypoderm', 'cell_death': False, 'isterminal': True}, {'lineage_id': 'ABarapaaapa', 'name': 'm1VR', 'type': 'muscle', 'cell_death': False, 'isterminal': True}]


## The below illustrates that the times don't match up

In [33]:
print(pos_time)
print(gene_time)

import numpy as np

overlap = np.intersect1d(pos_time, gene_time)
print(overlap)
print(len(overlap))


[465. 470. 475. 480. 485. 490. 495. 500. 505. 510. 515. 520. 525. 530.
 535. 540. 545. 550. 555. 560. 565. 570. 575. 580. 585. 595. 600. 610.
 615. 620. 630. 635. 640. 650. 665. 670. 675. 680. 685. 705. 710. 715.
 740. 745. 765. 770. 775.]
47


In [66]:
############################ KEEEEEEEEEEEEEEEEEEEEEPPPP #####################



import h5py
import pandas as pd
import numpy as np
import os

master_directory = cp.DATA_PATH

# Placeholder for the last read h5 file path
h5_file_path = None

# Iterate and find the last h5 file for demonstration
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)

if h5_file_path:
    with h5py.File(h5_file_path, 'r') as f:
        pos_rpz = f['pos_rpz'][()]  # Assuming shape is (3, num_cells, num_times)

    # Since the last dimension is the coordinates, we can reshape directly
    num_cells, num_times = pos_rpz.shape[0], pos_rpz.shape[1]
    # Flatten the array while keeping the r, p, z coordinates intact
    data_reshaped = pos_rpz.reshape(num_cells * num_times, 3)

    # Create DataFrame
    df_pos_rpz = pd.DataFrame(data_reshaped, columns=['r', 'p', 'z'])

    # Generate cell_id and time_id
    cell_ids = np.repeat(np.arange(num_cells), num_times)
    time_ids = np.tile(np.arange(num_times), num_cells)

    # Add cell_id and time_id to the DataFrame
    df_pos_rpz['cell_id'] = cell_ids
    df_pos_rpz['time_id'] = time_ids

    # Display the first few rows of the DataFrame
    print(df_pos_rpz.head())

          r         p          z  cell_id  time_id
0  3.092722 -1.208199  15.028050        0        0
1  3.148723 -1.970752  15.410750        0        1
2  4.094421 -2.825229  16.787140        0        2
3  3.943589 -0.301919  18.441759        0        3
4  2.134434 -2.099260  21.032261        0        4


In [67]:
########################## ADDING THE JSON INFO TO THE DATAFRAME ##############################

# Assuming cell_info and df_pos_rpz are already defined and populated as before

# Extend the cell_name_mapping to include additional attributes
cell_attributes_mapping = {i: {'name': cell['name'], 
                               'lineage_id': cell['lineage_id'], 
                               'type': cell['type'], 
                               'cell_death': cell['cell_death'], 
                               'isterminal': cell['isterminal']} 
                           for i, cell in enumerate(cell_info)}

# Function to safely get attributes for each cell_id
def get_cell_attribute(cell_id, attribute):
    return cell_attributes_mapping.get(cell_id, {}).get(attribute, 'Unknown')

# Now, map each cell_id in your DataFrame to its corresponding attributes
df_pos_rpz['lineage_id'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'lineage_id'))
df_pos_rpz['cell_type'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'type'))
df_pos_rpz['cell_death'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'cell_death'))
df_pos_rpz['isterminal'] = df_pos_rpz['cell_id'].apply(lambda x: get_cell_attribute(x, 'isterminal'))

# Now, df_pos_rpz includes additional columns for lineage_id, cell_type, cell_death, and isterminal
print(df_pos_rpz.head())


          r         p          z  cell_id  time_id  lineage_id cell_type  \
0  3.092722 -1.208199  15.028050        0        0  ABpraappaa  hypoderm   
1  3.148723 -1.970752  15.410750        0        1  ABpraappaa  hypoderm   
2  4.094421 -2.825229  16.787140        0        2  ABpraappaa  hypoderm   
3  3.943589 -0.301919  18.441759        0        3  ABpraappaa  hypoderm   
4  2.134434 -2.099260  21.032261        0        4  ABpraappaa  hypoderm   

  cell_death isterminal  
0      False       True  
1      False       True  
2      False       True  
3      False       True  
4      False       True  


In [63]:
# Assuming cell_info and df_pos_rpz are already defined and populated as before

# Create a mapping of cell_ids to cell names
# This assumes each cell name in cell_info should correspond to a cell_id in your DataFrame
cell_name_mapping = {i: cell['name'] for i, cell in enumerate(cell_info)}

# Now, map each cell_id in your DataFrame to its corresponding cell name
# If a cell_id does not have a corresponding entry in cell_name_mapping, use 'Unknown' or another placeholder
df_pos_rpz['cell_name'] = df_pos_rpz['cell_id'].apply(lambda x: cell_name_mapping.get(x, 'Unknown'))

# Now, df_pos_rpz includes a cell_name column
print(df_pos_rpz.head())

          r         p          z  cell_id  time_id cell_name
0  3.092722 -1.208199  15.028050        0        0      hyp4
1  3.148723 -1.970752  15.410750        0        1      hyp4
2  4.094421 -2.825229  16.787140        0        2      hyp4
3  3.943589 -0.301919  18.441759        0        3      hyp4
4  2.134434 -2.099260  21.032261        0        4      hyp4


In [65]:
# How many unknowns are there compared to known cell names?
num_unknowns = df_pos_rpz['cell_name'].value_counts().get('Unknown', 0)
num_knowns = df_pos_rpz.shape[0] - num_unknowns
print(f"Number of known cell names: {num_knowns}")
print(f"Number of unknown cell names: {num_unknowns}")


Number of known cell names: 103041
Number of unknown cell names: 3210


In [60]:
# how many unique cell ids are there in df_pos_rpz
num_unique_cells = df_pos_rpz['cell_id'].nunique()
print("Number of unique cells:", num_unique_cells)

Number of unique cells: 331


In [61]:
##### THIS FROM GPT #####

import h5py
import pandas as pd
import numpy as np
import json
import os

master_directory = cp.DATA_PATH

# Placeholder for the last read h5 file path
h5_file_path = None

# Iterate and find the last h5 file for demonstration
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)

if h5_file_path:
    with h5py.File(h5_file_path, 'r') as f:
        pos_rpz = f['pos_rpz'][()]  # Assuming shape is (num_cells, num_times)
        # Extract cellinf JSON data
        cellinf_json = f['cellinf'][()][0][0].decode('utf-8')  # Adjusted based on structure
        cell_info = json.loads(cellinf_json)

    # Since the last dimension is the coordinates, we can reshape directly
    num_cells, num_times = pos_rpz.shape[0], pos_rpz.shape[1]
    # Flatten the array while keeping the r, p, z coordinates intact
    data_reshaped = pos_rpz.reshape(num_cells * num_times, 3)

    # Create DataFrame
    df_pos_rpz = pd.DataFrame(data_reshaped, columns=['r', 'p', 'z'])

    # Generate cell_id and time_id
    cell_ids = np.repeat(np.arange(num_cells), num_times)
    time_ids = np.tile(np.arange(num_times), num_cells)

    # Map cell IDs to cell names
    cell_names = [cell['name'] for cell in cell_info]  # Assume cell_info is a list of dicts with 'name'
    # Extend cell names to match the length of cell_ids in the DataFrame
    extended_cell_names = np.repeat(cell_names, num_times)

    # Add cell_id, time_id, and cell names to the DataFrame
    df_pos_rpz['cell_id'] = cell_ids
    df_pos_rpz['time_id'] = time_ids
    df_pos_rpz['cell_name'] = extended_cell_names  # Add cell names

    # Display the first few rows of the DataFrame
    print(df_pos_rpz.head())


ValueError: Length of values (103041) does not match length of index (106251)

In [24]:
# Extract an example cell_id from that data
cell_id = 5
# extract a dataframe containing only that cell
df_cell = df_pos_rpz[df_pos_rpz['cell_id'] == cell_id]
# display iy
print(df_cell)

# Plot the max time id for each cell
df_max_time = df_pos_rpz.groupby('cell_id')['time_id'].max().reset_index()
df_max_time.columns = ['cell_id', 'max_time_id']
df_max_time

             r             p           z  cell_id  time_id
1605  3.059645 -1.232320e+00   15.193180        5        0
1606  3.064366 -1.940671e+00   15.495850        5        1
1607  4.152063 -2.815799e+00   17.506491        5        2
1608  3.916120 -3.043591e-01   18.507231        5        3
1609  2.030680 -2.124073e+00   20.972900        5        4
...        ...           ...         ...      ...      ...
1921  2.518113 -4.957089e-17  138.542496        5      316
1922  1.722124  1.846520e+00  138.499207        5      317
1923  1.167557 -1.557348e+00  139.856995        5      318
1924  1.429230  1.134767e+00  140.308594        5      319
1925  0.602170 -1.673056e+00  142.803299        5      320

[321 rows x 5 columns]


,cell_id,max_time_id
0,0,320
1,1,320
2,2,320
3,3,320
4,4,320
...,...,...
326,326,320
327,327,320
328,328,320
329,329,320


In [14]:
import h5py
import pandas as pd
import numpy as np


master_directory = cp.DATA_PATH

#  Iterate over the folders in the master directory
for folder in os.listdir(master_directory):
    folder_path = os.path.join(master_directory, folder)
    
    # Check if the folder is a directory
    if os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for file in os.listdir(folder_path):
            h5_file_path = os.path.join(folder_path, file)


# Open the h5 file
with h5py.File(h5_file_path, 'r') as f:
    # Extract the pos_rpz dataset
    pos_rpz = f['pos_rpz'][()]


# Display a preview of pos_rpz
print(pos_rpz)

# 

#    

# # Convert pos_rpz to a pandas dataframe
# df_pos_rpz = pd.DataFrame(pos_rpz)

# # Display the dataframe
# print(df_pos_rpz)


[[[  3.0927217   -1.20819914  15.02805042]
  [  3.14872289  -1.970752    15.41075039]
  [  4.09442139  -2.82522869  16.78713989]
  ...
  [  1.28960478  -1.46278107 136.0007019 ]
  [  1.49707925   1.14948523 136.72140503]
  [  0.66882074  -1.56147957 138.78819275]]

 [[  3.09667134  -1.21407139  15.05492973]
  [  3.14045668  -1.96326554  15.42298031]
  [  4.10371256  -2.81815958  16.90217972]
  ...
  [  1.2685194   -1.48638749 136.73500061]
  [  1.48197532   1.14719629 137.43559265]
  [  0.67123091  -1.58409417 139.59829712]]

 [[  3.09630632  -1.21882415  15.08452988]
  [  3.12669277  -1.9559716   15.43801022]
  [  4.11432838  -2.81393862  17.03248024]
  ...
  [  1.24445486  -1.50697112 137.49749756]
  [  1.46793652   1.14435792 138.15409851]
  [  0.66521484  -1.60240138 140.41499329]]

 ...

 [[  2.43339562  -1.61188483  19.32139969]
  [  2.43172789  -1.61731923  19.75414085]
  [  5.09780216  -2.8565743   44.52928162]
  ...
  [  0.9287312   -1.43783271 189.26359558]
  [  0.54483759   

# Import your experiment list

Check that the list generated in the next cell contains your conditions and replicates

In [2]:
# Get the experiment list from the experiments listed in the config 
exp_list = cp.populate_experiment_list()
display(exp_list)
print(cp.SAVED_DATA_PATH)

UnboundLocalError: local variable 'exp_list_df' referenced before assignment

<div class="alert alert-block alert-success">
<h2>2. Measurements of morphology and migration</h1>
</div>

### This cell does migration and morphology measurements for all of the cells at each timepoint 

In [ ]:
# Load, process and combine the dataframes (including segmentation and migration calculations)
comb_df = cp.combine_dataframes(exp_list)

comb_df, new_factors = cp.measurement_pipeline(comb_df, mixed=cp.MIXED_SCALING, factors_to_timeaverage = cp.ALL_FACTORS) 
display(new_factors)

# Returns a filtered dataframe, while also adding included column to comb_df
comb_df, filt_counts = cp.apply_filters(comb_df)

# Process a time-averaged DataFrame
tavg_df = cp.time_average(comb_df)

In [ ]:
# Use filt_df or comb_df depending on what you want to see
f=cp.multi_condition_timeplot(filt_df, chosen_factor)
f.show()

In [ ]:
# load the tavg_df from csv
tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv')

In [ ]:
MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient']

# Region property factors to be extracted from the cell contours
# This list must match with props from regionprops

REGIONPROPS_LIST = ['area',
                    'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    'extent',
                    'filled_area',
                    'major_axis_length',
                    'minor_axis_length',
                    'orientation',
                    'perimeter',
                     'solidity']

In [ ]:
f=cp.plots_of_differences_sns(tavg_df,factor='rip_L')
f.show()

And then gives you the option to save the dataframes into your automatically created 'saved data' folder

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    comb_df.to_csv(cp.SAVED_DATA_PATH + 'comb_df.csv', index=False)
    tavg_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv', index=False)

### Optional: load in precreated dataframes

In [ ]:
comb_df = pd.read_csv(cp.SAVED_DATA_PATH + 'comb_df.csv')
tavg_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tavg_df.csv')

### Optional: do filtering on the data (on top of what has been stated in the config file)

In [ ]:
# User-defined filters in dict {factor:(min, max)}

data_filters = {
#   "speed": (10, 100),
  "area": (50, 10000),
#    "frame": (0, 450), # Warning: range will change if self-normalized
#   "ntpts": (12,1800)
}

# Returns a filtered dataframe, while also adding included column to comb_df
filt_df, filt_counts = cp.apply_filters(comb_df,how='any', filter_dict=data_filters)

fig = cp.visualize_filtering(filt_df, filt_counts)

### Make a preview plot of any of these factors

In [ ]:
chosen_factor = 'area' # Enter in a factor from the list below

In [ ]:
## Factor list: 
all_factors =  ['area',
                'bbox_area',
                'eccentricity',
                'equivalent_diameter',
                'extent',
                'filled_area',
                'major_axis_length',
                'minor_axis_length',
                'orientation',
                'perimeter',
                'solidity',
                'euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient'
                'aspect',
                'rip_p',
                'rip_K',
                'rip_L']

In [ ]:
# Use filt_df or comb_df depending on what you want to see
f=cp.multi_condition_timeplot(filt_df, chosen_factor)
f.show()

# Plot all metrics

This cell makes comparative plots for every single metric and saves them in your output folder

* Plots of difference
* Timeplots of difference
* Marginal xy plots
* Simple bar plots
* Superplots - useful for comparing between replicates

<div class="alert alert-block alert-danger">
Check that you are happy with your extra filtering before continuing
Run the next cell on the filtered dataframe or the unfiltered dataframe once you are ready
</div>

In [ ]:
# Outputs plots of all metrics for all factors
cp.comparative_visualization_pipeline(comb_df, num_factors=all_factors) 

<div class="alert alert-block alert-success">
<h2>3. Definition of single timepoint behavioural clusters using UMAP and HDBSCAN</h1>
</div>

### Optional: for new datasets perform correlation analysis to understand which factors correlate to one another

This may aid in choosing the most important factors, aiding clustering

In [ ]:
df_in = comb_df
cp.correlation_matrix_heatmap(df_in, factors = cp.ALL_FACTORS)

### Optional: use variance thresholder for further insight

In [ ]:
chosen_dr_factors = cp.variance_threshold(comb_df, threshold_value=0.03)
chosen_dr_factors

### Optional: define a new list of dr_factors to use for UMAP

In [ ]:
REGIONPROPS_LIST = ['area',
                    'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    'extent',
                    'filled_area',
                    'major_axis_length',
                    'minor_axis_length',
                    'orientation',
                    'perimeter',
                     'solidity'
                     ]

MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                'orientedness', 
                'directedness',
                'turn_angle',
                'endpoint_dir_ratio',
                'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                'glob_turn_deg',
                'arrest_coefficient']

ADDITIONAL_FACTORS = ['aspect', 'rip_L'] # 'rip_p', 'rip_K', 


DR_FACTORS = REGIONPROPS_LIST + MIG_FACTORS + ADDITIONAL_FACTORS

## Perform UMAP and cluster analysis

### First, do UMAP, save the new df and plot the UMAP

Well separated clusters depend mostly on 1. the input factors and 2. the umap_nn setting

You can change both, depending on the nature of your data, in order to achieve a reasonable level of separation of clusters

In [ ]:
###### User alterable parameters ######
tsne_perp=150
umap_nn = 30 #umap nearest neighbours
min_dist = 0.0 #umap minimum distance (usually keep this at 0 or very low)
n_components = 3 # number of umap dimensions to calculate
#######################################



dr_df = cp.dr_pipeline_multiUMAPandTSNE(comb_df, 
                    dr_factors=DR_FACTORS,
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',) # A number of scaling methods are available: 'choice', 'minmax', 'standard', 'robust', 'normalize', 'quantile', 'maxabs', 'yeo-johnson', 'box-cox'

dr_df.to_csv(cp.SAVED_DATA_PATH + 'dr_df.csv', index=False) # Saves the df

cp.plot_3D_scatter(dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='condition', ticks=False, identifier='dr_df' + '_byCONDITION_',dotsize = 0.01, alpha=0.1, markerscale = 100) #color = label or condition  

### Second, identify clusters and exemplar cells using HDBSCAN

In [ ]:
##### User adjustable parameters #####
min_cluster_size = 1000
min_samples = 500
cluster_by = 'UMAPNDIM' # UMAPNDIM = default, clusters on UMAPs. NDIM = alternate, clusters on all dimensions
metric = 'euclidean' # See https://hdbscan.readthedocs.io/en/latest/api.html#hdbscan.HDBSCAN for options
#######################################

lab_dr_df, exemplar_df=cp.hdbscan_clustering(dr_df, min_cluster_size=min_cluster_size, min_samples=min_samples, cluster_by=cluster_by,  metric=metric)

lab_dr_df.name='lab_dr_df'
name = lab_dr_df.name

lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_dr_df.csv', index=False)
exemplar_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv', index=False)

cp.plot_3D_scatter(lab_dr_df_30, 'UMAP1', 'UMAP2', 'UMAP3', colorby='label', ticks=False, identifier=name + '_byCLUSTERID___',dotsize = 0.01, alpha=0.1, markerscale = 100) #color = label or condition   


In [ ]:
# cp.plot_3D_scatter_dev(exemplar_cell_tracks_df_500_gold, 'UMAP1', 'UMAP2', 'UMAP3', colorby='uniq_id', ticks=False, identifier='exemplar_cell_tracks_df_500_gold' + '_byCONDITION___',dotsize = 20, alpha=0.1, markerscale = 5) #color = label or condition  2
# cp.plot_3D_scatter_dev(tptlabel_dr_df, 'UMAP1', 'UMAP2', 'UMAP3', colorby='uniq_id', ticks=False, identifier='exemplar_cell_tracks_df_500_gold' + '_byCONDITION___',dotsize = 0.01, alpha=0.1, markerscale = 5) #color = label or condition  2

## Then plot the 'fingerprint' plot of percentage in each cluster per condition

In [ ]:
# This is the new combo
cluster_purity_df = cp.purity_pointsinclusterspercondition(lab_dr_df) 
display(cluster_purity_df)
f = cp.purityplot_percentcluspercondition(lab_dr_df, cluster_purity_df) 

### Optional: explore the clusters with interactive 3D plot

In [ ]:
cp.interactive_plot_3D_UMAP(df=lab_dr_df,colorby = 'Condition_shortlabel', symbolby = 'Condition_shortlabel', what = ' AllTimeUMAPwithclusters') # TavgUMAPwithclusters

### Optional: all other conditions colored grey, chosen condition in color

In [ ]:
df=lab_dr_df

condlist = df['Condition_shortlabel'].unique().tolist() #get unique list of conditions from df
print(condlist) # show the condition list
# chosen_condition = '' #specify a chosen condition from the list
chosen_condition = condlist[0] # or choose the first one
print(chosen_condition)

cp.interactive_plot_3D_UMAP_chosen_condition(df, chosen_condition, opacity_grey=0.01, marker_size_all=2,) #change opacity and marker size to suit the data

### Optional: make UMAP plots colored by metric contributors - the more intense the color, the higher the contribution the metric to a cluster

In [ ]:
# First one colors per metric
cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=lab_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
                                                   identifier='inferno', colormap='inferno', coloredbycondition = False, samplethedf = False)
#second one colors per condition
# cp.plot_UMAP_subplots_coloredbymetricsorconditions(df_in=tptlabel_dr_df, x= 'UMAP1', y= 'UMAP2', z = 'UMAP3', n_cols = 5, ticks=False, metrics = cp.ALL_FACTORS, scalingmethod='choice',
#                                                    identifier='inferno', colormap='inferno', coloredbycondition = True, samplethedf = False)

### Perform UMAP then HDBSCAN on the tavg_df

### at the moment, just do this step as it is needed for compatibility later on

In [ ]:
tsne_perp=150
umap_nn = 20#4#60
min_dist = 0.0 #0.15 
n_components = 3

tavg_dr_df = cp.dr_pipeline_multiUMAPandTSNE(tavg_df, 
                    dr_factors=new_DR_FACTORS,# new_DR_FACTORS # DR_FACTORS #only_tmeans # cp.DR_FACTORS
                    n_components = n_components,
                    umap_nn=umap_nn,
                    min_dist= min_dist,
                    scalingmethod = 'choice',) # log2minmax # powertransformer #minmax

lab_tavg_dr_df, exemplar_tavg_df=cp.hdbscan_clustering(tavg_dr_df, min_cluster_size=50,min_samples=50,cluster_by='UMAPNDIM',  metric='euclidean', plot=False) # 
 

<div class="alert alert-block alert-danger">
Save your dataframes so you can come back to this step if necessary
</div>

In [ ]:
OVERWRITE_DATAFRAMES = True

if OVERWRITE_DATAFRAMES:
    tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tavg_dr_df.csv', index=False)
    lab_tavg_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_dr_df.csv', index=False)
    exemplar_tavg_df.to_csv(cp.SAVED_DATA_PATH + 'exemplar_tavg_df.csv', index=False)

In [ ]:
#Run this function to put the labels into the lab_tavg_lab_dr_df. Slow function. Can update search by uniq_id alone...

lab_tavg_lab_dr_df=cp.add_tavglabel_todf(lab_dr_df, lab_tavg_dr_df)
lab_tavg_lab_dr_df.to_csv(cp.SAVED_DATA_PATH + 'lab_tavg_lab_dr_df.csv', index=False)

## Quantify the plasticity 

In [ ]:
tptlabel_dr_df = cp.count_cluster_changes_with_tavg(lab_tavg_lab_dr_df)
tptlabel_dr_df.to_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv', index=False)

### Plots of plasticity

In [ ]:
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')

In [ ]:
df=tptlabel_dr_df
# all='\_allcells'
cp.plot_plasticity_changes(df, identifier='\_allcells', maxy=4) #problem with NaNs in the data

In [ ]:
df=tptlabel_dr_df
cp.plot_plasticity_countplots(df, identifier='_allcells')

In [ ]:
df=tptlabel_dr_df
cp.plot_cumulative_plasticity_changes_main(df, identifier='\_allcells', miny=None, maxy=None, t_window_multiplier = cp.T_WINDOW_MULTIPLIER, plotallcells = False)

# Disambiguate the clusters

### First, choose a number of exemplar cells to pick out from the exemplar cell list to display

In [ ]:
# Choose a number of exemplars to display for each cluster
n=2
exemplar_df = exemplar_df.groupby('label').apply(lambda x: x.sample(min(n,len(x)))).reset_index(drop=True)

In [ ]:
size=220 #

df= tptlabel_dr_df #from the all analysis part
exemp_df=exemplar_df #from the cluster analysis part.

top_dictionary, contributions_df_singletpoints, scaled_df=cp.contribution_to_clusters(df_in=tptlabel_dr_df,  howmanyfactors=3, dr_factors= newnew_DR_FACTORS) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df, scaled_df)
result_df = cp.create_cluster_averages_table(top_dictionary, df, scaled_df)
cp.disambiguate_timepoint(df, exemp_df, scaled_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True, trajectory = False) 

In [ ]:

tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')
exemplar_df = pd.read_csv(cp.SAVED_DATA_PATH + 'exemplar_df.csv')

## Then, to visualize single cells with many timepoints, select cells with lots of timepoints

In [ ]:
#### User inputs ####
whole_df = tptlabel_dr_df
exemplar_df = exemplar_df
numberofdesiredtimepoints = int(whole_df['ntpts'].mean())
# numberofdesiredtimepoints = 200
numberofcellspercluster = 40
num_clusters_whole_dataset = len(whole_df['label'].unique())

override = int((numberofcellspercluster*num_clusters_whole_dataset)*0.7)
#####################

exemplar_df_filt, exemplar_cell_tracks_df = cp.filter_exemplars(whole_df=whole_df, exemplar_df = exemplar_df, numberofdesiredtimepoints = numberofdesiredtimepoints, 
                                                                    numberofcellspercluster = numberofcellspercluster, override = override)

In [ ]:
df=exemplar_cell_tracks_df
# cp.plot_cumulative_plasticity_changes_test2(df, identifier='\_exemplars_only_3_df__', miny=None, maxy=None, t_window_multiplier = 1, plotallcells = True) #deprecated, use the small multiples version
cp.plot_cumulative_plasticity_changes_main(df, identifier='\_exemplars_only_3_df__', miny=None, maxy=None, t_window_multiplier = 1, plotallcells = False)

### Plot any factor as small multiples from the exemplars

In [ ]:
df = exemplar_cell_tracks_df
whichcolumntoplot = 'label'

cp.plot_small_multiples(df, whichcolumntoplot)

### Re'disambiguate the new exemplar df!

In [ ]:
size=300 #

df= tptlabel_dr_df #from the all analysis part
exemp_df=exemplar_df #from the cluster analysis part.

top_dictionary, contributions_df_singletpoints, scaled_df=cp.contribution_to_clusters_topdictionary(df_in=tptlabel_dr_df,  howmanyfactors=10, dr_factors= newnew_DR_FACTORS) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df, scaled_df)
cp.disambiguate_timepoint_dev(df, exemp_df, scaled_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True) 

# Useful things

In [ ]:
REGIONPROPS_LIST = ['area',
                    # 'bbox_area',
                    'eccentricity',
                    'equivalent_diameter',
                    # 'extent',
                    'filled_area',
                    'major_axis_length',
                    'minor_axis_length',
                    # 'orientation',
                    'perimeter',
                    #  'solidity'
                     ]

MIG_FACTORS = ['euclidean_dist',     
                'cumulative_length', 
                'speed',
                # 'orientedness', 
                # 'directedness',
                # 'turn_angle',
                'endpoint_dir_ratio',
                # 'dir_autocorr',
                'outreach_ratio',
                'MSD',                
                'max_dist',           
                # 'glob_turn_deg',
                'arrest_coefficient']

ADDITIONAL_FACTORS = ['aspect', 'rip_L'] # 'rip_p', 'rip_K', 


newnew_DR_FACTORS = REGIONPROPS_LIST + MIG_FACTORS + ADDITIONAL_FACTORS

<div class="alert alert-block alert-success">
<h2>4. Trajectory measurement: Damerau-Levenshtein</h1>
</div>

In [ ]:
tptlabel_dr_df = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df.csv')

### First filter the tptlabel_dr_df to include only a subset of data of similar timescale

In [ ]:
low = 200
high = 220

tptlabel_dr_df_filt = tptlabel_dr_df[tptlabel_dr_df['ntpts'].between(low, high)]

### Verify that the filtered data reflects the total data

In [ ]:
factorchoice = 'speed'

In [ ]:
# Makes timeplots of the unfiltered and filtered data

f=cp.multi_condition_timeplot(tptlabel_dr_df, factorchoice)
f.show()
f=cp.multi_condition_timeplot(tptlabel_dr_df_filt, factorchoice)
f.show()

In [ ]:
# Plot of difference of the unfiltered and filtered data
f = cp.plots_of_differences_sns(tavg_df, factorchoice)
f.show()

In [ ]:
tavg_trajectory_df = cp.time_average(tptlabel_dr_df)
f = cp.plots_of_differences_sns(tavg_trajectory_df, factorchoice)
f.show()

### Perform Damerau-Levenshtein analysis

In [ ]:
df = tptlabel_dr_df_filt
distance_matrix_dameraulev = cp.calculate_edit_distances(df,distancemetric = 'dameraulev', print_interval=10000) #fastdtw # dameraulev # mongeelkan
print(distance_matrix_dameraulev.shape)

In [ ]:
# Save the distance matrix
# np.save(cp.SAVED_DATA_PATH + 'distance_matrix_dameraulev.npy', distance_matrix_dameraulev)

### Perform a UMAP/HDBSCAN parameter sweep, and select plots

In [ ]:
'''Sweep'''

df = tptlabel_dr_df_filt
for n_neighbors in [8, 10, 12]:
    for min_samples in [5,8,10, 15, 30, 40]:
        for min_cluster_size in [5,8,10, 15, 30, 40]:
            print(f'min_samples = {min_samples}')
            print(f'min_cluster_size = {min_cluster_size}')
            print(f'n_neighbors = {n_neighbors}')
            tptlabel_dr_df_filt_clusteredtrajectories = cp.cluster_sequences(df, distance_matrix_dameraulev,
             do_umap=True, eps=0.1, min_samples=min_samples, min_cluster_size=min_cluster_size, n_neighbors=n_neighbors)

In [ ]:
'''Chosen UMAP and HDBSCAN parameters'''

min_samples = 30
min_cluster_size = 20
n_neighbors = 12

df = tptlabel_dr_df_filt

print(f'min_samples = {min_samples}')
print(f'min_cluster_size = {min_cluster_size}')
print(f'n_neighbors = {n_neighbors}')
tptlabel_dr_df_filt_clusteredtrajectories = cp.cluster_sequences(df, distance_matrix_dameraulev,
 do_umap=True, eps=0.1, min_samples=min_samples, min_cluster_size=min_cluster_size, n_neighbors=n_neighbors)

Get the fingerprint plot of trajectories

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories

cluster_purity_df = cp.purity_pointsinclusterspercondition(df, cluster_label='trajectory_id') 
f = cp.purityplot_percentcluspercondition(df, cluster_purity_df, cluster_label='trajectory_id', dotsize = 30) 

 ### Disambiguate the trajectory clustered cells:
 1) Make an exemplar_df_trajectories containing example rows
 2) Get the full tracks from those rows and make exemplar_df_trajectories_fulltrack
 2) Disambiguate with exemplar_df_trajectories
 3) Plot multiples with exemplar_df_trajectories_fulltrack

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
exemplar_df_trajectories, exemplar_df_trajectories_fulltrack  = cp.make_exemplar_df_basedon_trajectories(df, cells_per_traj=6)

In [ ]:
# full_tracks_df = pd.read_csv(cp.SAVED_DATA_PATH + 'full_tracks_df.csv')
df = exemplar_df_trajectories_fulltrack
cp.plot_trajectories(df=exemplar_df_trajectories_fulltrack, global_y=True, global_x=True)

In [ ]:
size=300 #


df= tptlabel_dr_df_filt_clusteredtrajectories 
exemp_df=exemplar_df_trajectories 

top_dictionary, contributions_df_singletpoints, scaled_df=cp.contribution_to_clusters(df_in=tptlabel_dr_df,  howmanyfactors=2, dr_factors= newnew_DR_FACTORS) #BEFORE disambiguate_tavg(), then: lab_tavg_dr_df BEFORE disambiguate_timepoint(), then: #tptlabel_dr_df 
cp.plot_cluster_averages(top_dictionary, df, scaled_df)
result_df = cp.create_cluster_averages_table(top_dictionary, df, scaled_df)
cp.disambiguate_timepoint(df, exemp_df, scaled_df, top_dictionary=top_dictionary, XYRange=size,boxoff=True, trajectory = True) 

# Percent fingerprint plot for cluster IDs per TRAJECTORY

In [ ]:
# tptlabel_dr_df_filt_clusteredtrajectories = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df_filt_clusteredtrajectories_FINAL_10-12-2023.csv')

df = tptlabel_dr_df_filt_clusteredtrajectories
cp.fingerprintplot_clusters_per_trajectory(df)

# Plasticity of cells per trajectory

In [ ]:
tptlabel_dr_df_filt_clusteredtrajectories = pd.read_csv(cp.SAVED_DATA_PATH + 'tptlabel_dr_df_filt_clusteredtrajectories_FINAL_10-17-2023.csv')

In [ ]:
df=tptlabel_dr_df_filt_clusteredtrajectories
cp.plasticity_per_trajectory(df)

In [ ]:
df=tptlabel_dr_df_filt_clusteredtrajectories
# all='\_allcells'
cp.plot_plasticity_changes_trajectories(df, identifier='\_allcells', maxy=9 , t_window_multiplier = 1) #problem with NaNs in the data

# Animations of trajectories

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
cp.make_trajectory_animations(df, exemplar_df_trajectories, number_of_trajectories=2, colormode='cluster') # singlecluster, cluster, trajectory

Select a number of example cells from each trajectory ID to map back on to the data and display as stacks of PNGs

In [ ]:
number_of_trajectories = 10 # Select a number of trajectories to plot

df = tptlabel_dr_df_filt_clusteredtrajectories

trajectory_ids = df['trajectory_id'].unique()

uniq_id_choices_list = []

for trajectory_id_choice in trajectory_ids:
    # for each trajectory_id, get a list of possible uniq_ids from the df
    uniq_id_choices = tptlabel_dr_df_filt_clusteredtrajectories[tptlabel_dr_df_filt_clusteredtrajectories['trajectory_id']==trajectory_id_choice]['uniq_id'].values
    # Make sure each once is unique in that list
    uniq_id_choices = np.unique(uniq_id_choices)
    # choose a number of random uniq_ids from that list based on number_of_trajectories
    uniq_id_choices = np.random.choice(uniq_id_choices, number_of_trajectories)
    # append each choice to a list
    uniq_id_choices_list.append(uniq_id_choices)
# flatten the list
chosen_uniq_ids = [item for sublist in uniq_id_choices_list for item in sublist]
    
print(chosen_uniq_ids)

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
cp.make_png_behaviour_trajectories(df,chosen_uniq_ids,XYRange = 300, follow_cell = False, invert=False)

In [ ]:
df = tptlabel_dr_df_filt_clusteredtrajectories
cp.make_raw_cell_pngstacks(df,chosen_uniq_ids,XYRange = 220, follow_cell=False, invert=False)